In [4]:
# import required packages
import cv2
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation

from keras import optimizers
from keras.optimizers import Adam
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

from keras.utils import np_utils

In [5]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)


# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        'data/FER 2013 default/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical'
        )

# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        'data/FER 2013 default/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical'
        )

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [6]:
# create model structure
emotion_model = Sequential()

# the model so far outputs 3D feature maps (height, width, features)
emotion_model.add(Conv2D(32, kernel_size=(5, 5), activation='elu', input_shape=(48, 48, 1), padding='same', kernel_initializer='he_normal'))
emotion_model.add(BatchNormalization(name='batchnorm_1'))
emotion_model.add(Conv2D(64, kernel_size=(5, 5), activation='relu', padding='same', kernel_initializer='he_normal'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.4))

emotion_model.add(Conv2D(128, kernel_size=(5, 5), activation='elu', padding='same', kernel_initializer='he_normal'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(5, 5), activation='relu', padding='same', kernel_initializer='he_normal'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.4))

# this converts our 3D feature maps to 1D feature vectors
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='elu', kernel_initializer='he_normal'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=7,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
]


2023-04-03 14:35:01.580474: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/roma/anaconda3/envs/deep_learning/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [8]:
# Train the neural network/model
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=200,
        validation_data=validation_generator,
        validation_steps=7178 // 64,
        callbacks=callbacks)

Epoch 1/200


/var/folders/cb/nv527ffd4sn4f92077l2twqr0000gn/T/ipykernel_66650/131353881.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


448/448 [==============================] - 270s 602ms/step - loss: 2.0028 - accuracy: 0.2549 - val_loss: 1.6482 - val_accuracy: 0.3467 - lr: 1.0000e-04
Epoch 2/200
448/448 [==============================] - 264s 590ms/step - loss: 1.7086 - accuracy: 0.3178 - val_loss: 1.5701 - val_accuracy: 0.3934 - lr: 1.0000e-04
Epoch 3/200
448/448 [==============================] - 267s 595ms/step - loss: 1.6429 - accuracy: 0.3560 - val_loss: 1.5449 - val_accuracy: 0.4081 - lr: 1.0000e-04
Epoch 4/200
448/448 [==============================] - 270s 602ms/step - loss: 1.5995 - accuracy: 0.3712 - val_loss: 1.6103 - val_accuracy: 0.4180 - lr: 1.0000e-04
Epoch 5/200
448/448 [==============================] - 271s 606ms/step - loss: 1.5563 - accuracy: 0.3927 - val_loss: 1.6206 - val_accuracy: 0.4210 - lr: 1.0000e-04
Epoch 6/200
448/448 [==============================] - 276s 616ms/step - loss: 1.5218 - accuracy: 0.4062 - val_loss: 1.4621 - val_accuracy: 0.4616 - lr: 1.0000e-04
Epoch 7/200
448/448 [=======

In [9]:
# save model structure in jason file
model_json = emotion_model.to_json()
with open("emotion_model_3.2_epochs77.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model_3.2_epochs77.h5')